In [1]:
import numpy as np
from scipy.stats import gumbel_r
import matplotlib.pyplot as plt 
import scipy.integrate as integrate
import math
from scipy.stats import norm
import seaborn as sns
import scipy.stats as stats


In [2]:
#Derivative of log likelihood, for Newton-Raphson method
def l_dash(y, n, sigma, mu = 14.9787):
  term1 = -1*n/sigma
  term2 = np.sum((y - mu)/(sigma ** 2))
  term3 = -1*np.sum(((y - mu)/(sigma ** 2)) * np.exp(-1*(y - mu)/sigma))
  l_dash = term1 + term2 + term3

  return l_dash

In [3]:
#Double derivative of log likelihood, for Newton-Raphson method
def l_double_dash(y, n, sigma, mu = 14.9787):
  term1 = n/(sigma ** 2)
  term2 = -2*np.sum(y - mu)/(sigma ** 3)
  term3 = 2*np.sum((y - mu) * np.exp(-1*(y - mu)/sigma))/(sigma ** 3)
  term4 = -1*np.sum((y - mu)**2 * np.exp(-1*(y-mu)/sigma))/(sigma**4)
  l_double_dash = term1 + term2 + term3 + term4

  return l_double_dash 

In [4]:
#Estimating sigma using Newton-Raphson method
def newton_raphson(y, n, threshold = 1e-4):
  sigma = 0.1 #Initial value of sigma
  
  while(l_dash(y, n, sigma) > threshold):
    sigma = sigma - l_dash(y, n, sigma)/l_double_dash(y, n, sigma)   #Newton-Raphson method update equation
  
  return sigma

In [5]:
def estimate_sigma(n):
  mean = 5
  K = 20
  
  X = np.random.exponential(mean, (K, n))   #generating 20 samples from exponential distribution
  Y = np.max(X, axis = 0)   #taking its maximum (Gumbel distribution)
  sigma = newton_raphson(Y, n)  #Estimating sigma using Newton-Raphson method
  
  return sigma

In [ ]:
N = [1, 10, 100, 1000, 10000] #list of values of N
runs = 1000  #no. of runs
sigmas = np.zeros((runs, len(N))) 

#estimating sigma
for j, n in enumerate(N):
  for i in range(runs):
    sigmas[i, j] = estimate_sigma(n)

In [ ]:
#printing sigma for a single run
for n in N:
  sigma_hat = estimate_sigma(n)
  print("n: ", n, "sigma_est: ", sigma_hat)

In [ ]:
var = np.zeros(len(N))
expec = np.zeros(len(N))

#printing estimation of sigma and variance for 100 runs
for j,n in enumerate(N):
  var[j] = np.var(sigmas[:, j])
  expec[j] = np.mean(sigmas[:, j])
  print("N: ", n, "Sigma: ", expec[j], "Variance: ", var[j])

In [ ]:
mu = 14.9787
sigma = 5

x=np.arange(0,10,0.001)
probs=np.array(range(runs))/float(runs)
for j,n in enumerate(N):
    sortVal = np.sort(sigmas[:,j])
    plt.plot(sortVal, probs,label="n={n}".format(n=n))
plt.legend()
plt.title("CDF for various values of N")
plt.xlim([4,6])
plt.title("CDF of $\hat{\sigma}_{gev}$ for various values of N")
plt.xlabel("$\hat{\sigma}_{gev}$")
plt.ylabel("CDF")
plt.show()

In [ ]:
#calculating Fisher information

def integrand(y, sigma = 5, mu = 14.9787):
  term1 = 1/(sigma ** 2)
  term2 = -2*(y - mu)/(sigma ** 3)
  term3 = 2*((y - mu) * np.exp(-1*(y - mu)/sigma))/(sigma ** 3)
  term4 = -1*((y - mu)**2 * np.exp(-1*(y-mu)/sigma))/(sigma**4)
  l_double_dash = term1 + term2 + term3 + term4

  return l_double_dash

def pdf(y):
  return gumbel_r.pdf(y, 14.9787, 5)

I = integrate.quad(lambda y: integrand(y)*pdf(y), 0, np.inf)
fisher = -1*I[0]

In [ ]:
#cdf with fisher
s=1
x=np.arange(-10,10,0.01)                 

nvalues=np.empty(np.shape(sigmas))
for j,n in enumerate(N):
  for i in range(runs):
    nvalues[i,j]=n

probs=np.array(range(runs))/float(runs)
for j,n in enumerate(N):
    sig_ecor[:,j] = math.sqrt(n) * (sigmas[:, j] - expec[j])
    sortVal = np.sort(sig_ecor[:,j])
    plt.plot(sortVal, probs,label="n={n}".format(n=n))
plt.plot(x,stats.norm.cdf(x, 0, np.sqrt(1/fisher)),'r.', label="normal with fisher")
plt.xlim([-10,10])
plt.title("CDF of $\sqrt{N}*(\hat{\sigma}_{gev} - \sigma)$")
plt.xlabel("y")
plt.ylabel("Density")
plt.legend()
# plt.savefig('3_2.png')
plt.show()

In [ ]:
#plotting pdf


width=0.006
b=np.arange(0,10,width)
for j,n in enumerate(N):
    if n!=0:
        probs, b = np.histogram(sigmas[:,j], bins=b) # bin it into n = N//10 bins
        probs=probs/runs
        bincentre = b[:-1] + (b[1] - b[0])/2   # convert bin edges to centers
        plt.plot(bincentre, probs,label="n={n}".format(n=n))
        # plt.bar(bincentre, probs, align = 'center', width = width,label="n={n}".format(n=n))
plt.xlim([4.5,5])
plt.title("pdf of $\hat{\sigma}_{gev}$ various N")
plt.xlabel("$\hat{\sigma}_{gev}$")
plt.ylabel("Density")
plt.legend()
plt.show()

In [ ]:
#plotting sqrt(N)*(sigma_est - sigma) and the normal distribution
for j,n in enumerate(N):
  x = math.sqrt(n) * (sigmas[:, j] - expec[j])
  sns.kdeplot(x, label = "n={n}".format(n=n), legend = True)
y = np.linspace(-15, 20)
plt.plot(y, norm.pdf(y, 0, math.sqrt(1/fisher)), label = "Normal dist")
plt.title("$\sqrt{N}*(\hat{\sigma}_{gev} - \sigma)$ and the normal distribution")
plt.xlabel("x")
plt.legend()